In [43]:
# import openpyxl module 
import openpyxl 
import re
  
# Give the location of the file 
path = "/Users/mokrzesik/Desktop/Michael/General/Disney/05-01-2024 Disney Dining.xlsx"

# set the sheet names of the workbook
sheet1 = 'Restaurant List'
sheet2 = 'Menus'
  
# To open the workbook 
# workbook object is created 
wb = openpyxl.load_workbook(path) 
  
# Get menu workbook
# from the active attribute 
menu = wb.worksheets[1]

In [131]:
decimal_pattern = re.compile(r'[-+]?\d*\.\d+')
dollar_sign = re.compile(r'\$')
market_price = re.compile(r'market price', re.IGNORECASE)
number_pattern = re.compile(r'\d+')
date_pattern = re.compile(r"['’‘]\d+")

def value_check(string):
  # checks if there is a dollar sign
  if dollar_sign.search(string):
    return True
  elif date_pattern.search(string):
    return False
  elif decimal_pattern.search(string) or number_pattern.search(string):
    return True
  else:
    return False
  

search_columns = ['A', 'D']

In [133]:
for i, cell in enumerate(menu['A']):
  # if the cell is empty, check if the D column has a number
  if date_pattern.search(cell.value):
    print(cell.value)

Leeuwin Estate, Art Series, Margaret River, Australia ‘19
Cage, Napa Valley, California ‘17
Fisher Vineyards, Mountain Estate Vineyard, Sonoma County, California ‘19
Far Niente, Napa Valley, California ‘20
Double-Zero, VGW, Willamette Valley, Oregon ‘17
Baumard, Clos du Papillon, Savennières, France ‘16
Selbach-Oster, *** Zeltinger Sonnenuhr, Germany ‘01
Domaine Weinbach, Schlossberg Grand Cru, Alsace France ‘19
Trimbach, Clos Ste Hune, Alsace, France ‘16
Knights Bridge, Knights Valley, California ‘18
Klein Constantia, Vin de Constance, Constantia, South Africa ‘16
Jean-Louis Chave, L’Hermitage, Hermitage, France ‘19
Avancia, Nobleza, Valdeorras, Spain ‘20 
Ktima Gerovassiliou, Epanomi, Greece ‘21
Tyrrell’s, Vat 1, Hunter Valley, Australia ‘15
Château d’Yquem, Sauternes, France ‘05
Borgogno, Derthona, Colli Tortonesi, Italy ‘19
Lopez de Heredia, Vina Gravonia, Crianza, Rioja, Spain ‘11
Chateau Grillet, Condrieu, France ‘18
Mouton-Rothschild, Aile d’Argent, Bordeaux, France ‘18
Pierre M

In [134]:
for i, cell in enumerate(menu['A']):
  # if the cell is empty, check if the D column has a number
  if value_check(cell.value):
    print(cell.value)
  
    

Sho Chiku Bai Sake - 6oz
Hana Fuji Apple Flavored Sake - 6oz
Sho Chiku Bai Junmai Nigori Sake - 300 mL
Sho Chiku Bai Sake - 6oz
Hana Fuji Apple Flavored Sake - 6oz
Sho Chiku Bai Junmai Nigori Sake - 300 mL
Line 39 Cabernet Sauvignon
Bulleit 95 Rye Whiskey
Frozen Novelties ($5.39- $5.99)
Assorted Beer ($8.50-$12.75)
Wine ($11.00-$16.00)
Hurricane 15oz
Yardstick 32oz
Kids Cup 17oz
$54 per adult, plus tax and gratuity
$34 per child, plus tax and gratuity
French 75 
$66 per adult, plus tax and gratuity
$41 per child, plus tax and gratuity
French 75 
Frosé (21+)
Vanilla Guinness Float (21+)
Mimosa (21+)
Vivoli Spritz (21+)
Blueberry Mimosa (21+)
Guinness Tiramisu (21+)
Kronenbourg 1664
Mini Babybel® Snack Cheese (3-pack)
Cuties® Mandarin (3-pack)
American 75
81 Bay Brewing Irish Red Ale
The King’s Experience 1oz Each
Broadbent Malmsey 10-year, Madeira, Portugal NV
Taylor Fladgate 20yr Tawny Port, Douro, Portugal NV
Hétszolo, 6 puttonyos, Tokaji, Hungary ‘10 - (3oz. pour - $30)
Alvear, 1927,

In [50]:
# loop through every row in the B column of menu
for i, cell in enumerate(menu['B']):
  # find if the cell is empty
  if cell.value == None:
    
    # if the cell is empty, check if the D column has a number
    if check_decimal_or_dollar(menu[f'D{i+1}'].value) or check_decimal_or_dollar.search(menu[f'D{i+1}'].value) or check_decimal_or_dollar.search(menu[f'D{i+1}'].value)
      or :
      # if it does, then take the number and add it do the B column
      print()
    elif market_price.search(menu[f'A{184}'].value):
      cell.value = 'Market Price'
    elif 
    
  

SyntaxError: invalid syntax (1956757368.py, line 7)